In [1]:
from aip import AipNlp
import pandas as pd
import time
import synonyms

smart_open library not found; falling back to local-filesystem-only
smart_open library not found; falling back to local-filesystem-only


Building prefix dict from C:\ProgramData\Anaconda3\lib\site-packages\synonyms\data\vocab.txt ...
I0814 10:43:48.485127 10160 __init__.py:111] Building prefix dict from C:\ProgramData\Anaconda3\lib\site-packages\synonyms\data\vocab.txt ...
Loading model from cache C:\Users\GWT\AppData\Local\Temp\jieba.ua5170a0f831c846f08094f57e4bca895.cache
I0814 10:43:48.488129 10160 __init__.py:131] Loading model from cache C:\Users\GWT\AppData\Local\Temp\jieba.ua5170a0f831c846f08094f57e4bca895.cache


>> Synonyms load wordseg dict [C:\ProgramData\Anaconda3\lib\site-packages\synonyms\data\vocab.txt] ... 


Loading model cost 1.508 seconds.
I0814 10:43:49.994206 10160 __init__.py:163] Loading model cost 1.508 seconds.
Prefix dict has been built succesfully.
I0814 10:43:49.995206 10160 __init__.py:164] Prefix dict has been built succesfully.


>> Synonyms on loading stopwords [C:\ProgramData\Anaconda3\lib\site-packages\synonyms\data\stopwords.txt] ...
>> Synonyms on loading vectors [C:\ProgramData\Anaconda3\lib\site-packages\synonyms\data\words.vector] ...


In [48]:
""" 你的 APPID AK SK """
APP_ID = '11653919'
API_KEY = 'UvzTwOdwV2viviQ71VoEOigz'
SECRET_KEY = 'RbdwB9RvAAmv3jc4qRyPizeQGY7gWgSt '
client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

In [49]:
options = {}
options["type"] = 12

In [50]:
f = open('jd行车记录仪评论2018-08-09.csv',encoding='utf-8',errors='ignore')
df = pd.read_csv(f)

In [51]:
content = df['content']

In [52]:
def geteachtextResult(commentList,errorlen = 0):
    banlist = []
    errorlist = []
    result = []
    print('共有%d条评论'%len(commentList))
    for text in commentList:
        nlp = client.commentTag(text,options)
        print('#',end='')
        if 'error_msg' in nlp:
            if nlp['error_code'] == 282130:
#                 print('不认识')
                errorlist.append(text)
            elif nlp['error_code'] == 18:
#                 print('ban')
                banlist.append(text)
            else:
                 print('未知错误')
                 print(nlp)
        elif 'items' in nlp:
            result.extend(nlp['items'])
        else:
            print('没报错但是也没结果')
            print(nlp)
    if len(banlist)!= 0:
#         a = copy.deepcopy(result)
        result.extend(geteachtextResult(banlist,errorlen+len(errorlist)))
        return result
#         return result.extend(geteachtextResult(banlist,errorlen+len(errorlist),result))
    else:
        print("\n分析完成！%d条没有结果"%(errorlen))
#         a = copy.deepcopy(result)
        return result

In [54]:
testComment = content[50:]
time1 = time.time()
com = geteachtextResult(testComment)
time2 = time.time()
print(str(time2-time1))

共有59998条评论
#######################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

UnicodeEncodeError: 'gbk' codec can't encode character '\ufffd' in position 53: illegal multibyte sequence

In [11]:
result = pd.DataFrame(com)

In [13]:
count = result['prop'].value_counts()
count

感觉       42
质量        7
东西        6
手机        5
效果        5
功能        5
价格        4
物流        4
外观        3
商品        3
种类        3
速度        3
看着        2
做工        2
性价        2
总体        2
快递        2
速度快       2
屏幕        2
效果不错      2
确实        2
款式不错      1
专业        1
物流速度      1
都会        1
超市        1
样子        1
外形        1
品牌不错      1
东西质量      1
产品性价比     1
评价        1
发货        1
画面        1
清晰度       1
包装        1
性价比       1
温度        1
狗         1
质量不错      1
品质        1
产品        1
分辨率       1
产品质量      1
发货速度      1
质感        1
服务        1
Name: prop, dtype: int64

In [26]:
#同义词
array=list(count.index)
synos=['0']

In [27]:
for i in range(len(array)):
    for j in range(len(synos)):
        if synonyms.compare(array[i], synos[j], seg=True)>0.8:
            break
        else:
            if j == (len(synos)-1):
                synos.append(array[i])
synos.remove('0')

C:\ProgramData\Anaconda3\lib\site-packages\synonyms\utils.py:246: RuntimeWarning: invalid value encountered in true_divide
  cosine = lambda a, b: dot(a, b)/(norm(a)*norm(b))
W0813 16:56:00.180377  8796 synonyms.py:157] not exist in w2v model: 性价
W0813 16:56:00.181377  8796 synonyms.py:157] not exist in w2v model: 性价
W0813 16:56:00.183380  8796 synonyms.py:157] not exist in w2v model: 性价
W0813 16:56:00.184381  8796 synonyms.py:157] not exist in w2v model: 性价
W0813 16:56:00.186383  8796 synonyms.py:157] not exist in w2v model: 性价
W0813 16:56:00.187382  8796 synonyms.py:157] not exist in w2v model: 性价
W0813 16:56:00.189385  8796 synonyms.py:157] not exist in w2v model: 性价
W0813 16:56:00.190385  8796 synonyms.py:157] not exist in w2v model: 性价


In [28]:
#好中坏
count=pd.DataFrame(count)

sy_count = pd.DataFrame()
sy_count['prop']=list(count.index)
sy_count['sum']=list(count['prop'])
sy_count['good']=0
sy_count['normal']=0
sy_count['bad']=0

In [29]:
for i in range(len(result)):
    for j in range(len(count)):
        if result['prop'][i]==sy_count['prop'][j]:
            if result['sentiment'][i]==2:
                sy_count['good'][j]=sy_count['good'][j]+1
            if result['sentiment'][i]==1:
                sy_count['normal'][j]=sy_count['normal'][j]+1
            if result['sentiment'][i]==0:
                sy_count['bad'][j]=sy_count['bad'][j]+1
            break

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [30]:
#同义词替代
for i in range(len(sy_count)):
    for k in range(len(synos)):
        if synonyms.compare(sy_count['prop'][i], synos[k], seg=True)>0.8:
            sy_count['prop'][i]=synos[k]
            break

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
W0813 16:56:10.022011  8796 synonyms.py:157] not exist in w2v model: 性价
W0813 16:56:10.024025  8796 synonyms.py:157] not exist in w2v model: 性价
W0813 16:56:10.025014  8796 synonyms.py:157] not exist in w2v model: 性价
W0813 16:56:10.027024  8796 synonyms.py:157] not exist in w2v model: 性价
W0813 16:56:10.029017  8796 synonyms.py:157] not exist in w2v model: 性价
W0813 16:56:10.030017  8796 synonyms.py:157] not exist in w2v model: 性价
W0813 16:56:10.031031  8796 synonyms.py:157] not exist in w2v model: 性价


In [31]:
#去重 排序
sy_count=sy_count.groupby('prop').sum()
sy_count=sy_count.sort_values(by="sum" , ascending=False)

In [32]:
sy_count

,sum,good,normal,bad
prop,,,,
感觉,43,41,0,2
质量,13,12,0,1
东西,12,11,0,1
价格,10,9,0,1
手机,7,7,0,0
效果,5,5,0,0
功能,5,5,0,0
物流,4,4,0,0
种类,3,1,0,2
